# Merging East River Tree Stem Geolocation Points
**Author:** 'Marshall Worsham' <br>
**Creation Date:** '09/21/2020' <br>
**Revision Date:** '12/22/2020' <br>

---

## Contents

1 - [Front matter](#front)<br>
2 - [Libraries](#libraries)<br>
3 - [Import reference table](#import)<br>
4 - [Exploratory analysis](#eda)<br>
5 - [Rename and move](#rename)<br>
6 - [Prepare for append](#prep)<br>
7 - [Append](#append)<br>

---


## Front matter<a id='front'></a>

This notebook contains markdown and code for post-processing point shapefiles generated from Trimble Geo7X GPS acquisitions in the East River domain. The script appends the `Site` name and `subdirectory` to each shapefile name, then selects all projected point shapefiles, groups them by `Site` name, and merges points from the same site. The result is a set of shapefiles containing tree geolocation points, one set for each site in the watershed where stem geolocations were acquired from 2018–2020. Most output files contain some extraneous points marking corners and errata, which are cleaned out in '00_EastRiver_Clean_Tree_GPSPoints.ipynb'. 

The script was developed in `Python 3.8.2` on a Macbook Pro 2014 running OSX 10.14.6.


## Libraries<a id='libraries'></a>

In [3]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import re
from matplotlib import pyplot as plt
from os.path import join, getsize
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

Define the working directory and list contents

In [12]:
os.getcwd()
directory = os.sep.join(['/Users', 'hmworsham', 'Desktop', 'RMBL', 'Projects', 'Watershed_Spatial_Dataset'])
source_dir = os.sep.join([directory, 'Source'])
out_dir = os.sep.join([directory, 'Output'])
gps_dir = os.sep.join([source_dir, 'RMBL_GPS_Data_All'])
os.listdir(gps_dir)[0:10]

['WORSHAMM081312A',
 'WORSHAMM072710A',
 'WORSHAMM080508A',
 'WORSHAMM081908B',
 'WORSHAMM081209A',
 'WORSHAMM081714B',
 'WORSHAMM080711A',
 'KUEPPERSL09306P',
 'WORSHAMM081815A',
 'WORSHAMM081013A']

## Import reference table<a id='reference'></a>
First we import a CSV describing filenames and associated sites. Then we slice to create a simple list of filenames and the site at which the data inside those files were acquired.


In [14]:
gps_index = pd.read_csv(os.sep.join([source_dir, 'EastRiver_GPS_Data_Index.csv']))
gps_index.loc[:, 'Filename':'Site'].head(10)

,Filename,Site
0,BAGSHAW_CARBONA,Carbon-2
1,BAGSHAW_CARBONB,Carbon-2
2,BAGSHAW_CARBONC,Carbon-2
3,BAGSHAW_CARBOND,Carbon-2
4,BAGSHAW_PL2A,PointLookout-2
5,BAGSHAW_PL2B,PointLookout-2
6,BAGSHAW_PL2C,PointLookout-2
7,BAGSHAW_PL2D,PointLookout-2
8,HETTEMAS082507A,PointLookout-1
9,HETTEMAS082508A,PointLookout-1


## Exploratory analysis<a id='eda'></a>
Some simple exploratory analysis reveals see how many unique files are associated with each site.

In [15]:
gps_index.groupby('Site').count()['Filename']

Site
Carbon-1           1
Carbon-2           4
OhioPass-1        12
PointLookout-1     8
PointLookout-2     4
ResearchMeadow     2
Schofield-19      14
Schofield-23       8
Schofield-24      11
Schofield-4        8
Schofield-5       13
Snodgrass-1       11
Snodgrass-2       19
SplainsGulch-1    13
Wash-1            12
Name: Filename, dtype: int64

In [16]:
# Scratch to set up the syntax for the function below that will relate filenames in the directory to filenames and site associations in the index dataframe
gps_index_sites = gps_index.loc[:,'Filename':'Site']
gps_index_sites.loc[gps_index_sites['Filename'] == 'WORSHAMM071610A'].iloc[0, 1]

'Snodgrass-2'

List the filenames in all subdirectories of `directory` by walking the subdirectories and string-splitting on the last `/` in the path to isolate filenames. 

In [20]:
for subdir, dirs, files in os.walk(gps_dir):
    for filename in files:
        subdir_name = subdir.rsplit('/', 1)[-1]
        print(subdir_name)

RMBL_GPS_Data_All
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM081312A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM072710A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM080508A
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081908B
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHAMM081209A
WORSHA

## Rename and move<a id='rename'></a>
The function below finds the name of the `subdirectory` that each shapefile lives in and finds the `Site` with which that subdirectory is associated. The function renames each shapefile by appending the `subdirectory` name and `Site` name to the filename, then moves all files into a single directory.

In [22]:
gps_dir

'/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Source/RMBL_GPS_Data_All'

In [27]:
for subdir, dirs, files in os.walk(gps_dir):
    for filename in files:
        gps_index_sites = gps_index.loc[:, 'Filename':'Site']
        subdir_name = subdir.rsplit('/', 1)[-1]
        index_sitename = str(gps_index_sites.loc[gps_index_sites['Filename'] == subdir_name, 'Site'].values).strip("[]").strip("'")
        newname = subdir_name + '_' + index_sitename + '_' + filename
        oldpath = subdir + os.sep + filename
        newpath = subdir + os.sep + newname
        print(oldpath)
        print(newname)
        print(newpath)
        os.rename(oldpath, newpath)
        if not re.search('Line.+', filename) and not re.search('Area.+', filename) and not re.search('Icon.+', filename):
            print(filepath)

/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Source/RMBL_GPS_Data_All/.DS_Store
RMBL_GPS_Data_All__.DS_Store


NameError: name 'filepath' is not defined

## Prepare for append<a id='prep'></a>

In [10]:
# list all files
renamed_dir = '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED'
allfiles = os.listdir(renamed_dir)
allfiles[0:10]
#allfiles

['BAGSHAW_CARBOND_Carbon-2_Point_ge.prj',
 'WORSHAMM072407A_SplainsGulch-1_Student_Project.shp.xml',
 'WORSHAMM081210A_OhioPass-1_Student_Project.shp.xml',
 'POWELLT082709A-TRAILPOINTS_Schofield-19_Line_gen_Project.cpg',
 'WORSHAMM080610A_Wash-1_Student.dbf',
 'WORSHAMM072710A_SplainsGulch-1_Student.shx',
 'WORSHAMM080914A_Schofield-5_Student_Project.sbn',
 'WORSHAMM081414A_Snodgrass-2_Student.shx',
 'WORSHAMM080916A_Schofield-5_Student.shx',
 'WORSHAMM081912A_Schofield-23_Student_Project.sbn']

In [11]:
# generate a list of unique site names represented in the dataset
sitelist = gps_index['Site'].unique().tolist()
sitelist

['Carbon-2',
 'PointLookout-2',
 'PointLookout-1',
 'Snodgrass-2',
 'Snodgrass-1',
 'SplainsGulch-1',
 'Carbon-1',
 'ResearchMeadow',
 'Schofield-24',
 'Schofield-19',
 'Schofield-23',
 'Wash-1',
 'Schofield-4',
 'OhioPass-1',
 'Schofield-5']

In [12]:
# filter only shapefiles containing point data types in the correct projection
# the target filenames will contain the tag "Project" AND either of the strings "Student" or "Point"
# filenames without "Student" and filenames containing "Area" and "Line" strings will be filtered out
point_str = 'Point_'
stud_str = 'Student'
proj_str = '_Project'
sf_allpoint = [i for i in allfiles if ((point_str in i or stud_str in i) and proj_str in i)]
print(len(allfiles))
print(len(sf_allpoint))
sf_allpoint

2019
1128


['WORSHAMM072407A_SplainsGulch-1_Student_Project.shp.xml',
 'WORSHAMM081210A_OhioPass-1_Student_Project.shp.xml',
 'WORSHAMM080914A_Schofield-5_Student_Project.sbn',
 'WORSHAMM081912A_Schofield-23_Student_Project.sbn',
 'WORSHAMM072409A_SplainsGulch-1_Student_Project.shp.xml',
 'WORSHAMM080713A_Schofield-5_Student_Project.sbx',
 'WORSHAMM081008A_Schofield-24_Student_Project.sbx',
 'WORSHAMM081815A_Schofield-19_Student_Project.shx',
 'WORSHAMM072912A_Snodgrass-1_Student_Project.sbx',
 'WORSHAMM080111A_Schofield-19_Point_ge_Project.shx',
 'WORSHAMM080609A_Wash-1_Student_Project.sbx',
 'WORSHAMM081609A_Schofield-5_Student_Project.dbf',
 'POWELLT082616A-BOUND-TEST1_Schofield-19_Point_ge_Project.sbn',
 'WORSHAMM072911A_Snodgrass-1_Student_Project.dbf',
 'WORSHAMM080511A_Wash-1_Student_Project.sbn',
 'WORSHAMM081515A_1_OhioPass-1_Student_Project.sbn',
 'WORSHAMM080614A_Schofield-4_Student_Project.shp.xml',
 'WORSHAMM081914A_Schofield-4_Student_Project.sbx',
 'WORSHAMM072211A_SplainsGulch-1_S

In [13]:
# manipulate the gps_index dataframe
notcorners = gps_index[~gps_index['Contents'].str.contains('corner')] # filter out names of subdirs containing corners
notcorners = notcorners['Filename'].to_list()
print(notcorners[:10])
print(len(sf_allpoint))
print(len(notcorners))

['HETTEMAS082508A', 'HETTEMAS082509A', 'HETTEMAS082510A', 'KUEPPERSL101610A', 'POWELLT082413A-TREES', 'POWELLT082506A-MEADOWTREES', 'POWELLT082509A-SCHO_24', 'POWELLT082510A', 'POWELLT082510B-SCHO_24', 'POWELLT082609A-SCH_24']
1128
111


In [14]:
# find files representing trees only, with .shp extension
trees_allfiles = [i for i in sf_allpoint if any(ii in i for ii in notcorners)]
trees_sf = [t for t in trees_allfiles if t.endswith('.shp')]

In [15]:
print(len(trees_sf))
print(trees_sf[:10])

112
['WORSHAMM071811A_Snodgrass-2_Student_Project.shp', 'HETTEMAS082510A_PointLookout-1_Student_Project.shp', 'WORSHAMM072909A_Snodgrass-1_Student_Project.shp', 'WORSHAMM081210A_OhioPass-1_Student_Project.shp', 'WORSHAMM081211B_OhioPass-1_Student_Project.shp', 'WORSHAMM072211A_SplainsGulch-1_Student_Project.shp', 'WORSHAMM081815A_Schofield-19_Student_Project.shp', 'WORSHAMM081908B_Schofield-23_Student_Project.shp', 'POWELLT082609A-SCH_24_Schofield-24_Point_ge_Project.shp', 'POWELLT082912A-403TRAILPOINTS_Schofield-23_Point_ge_Project1.shp']


## Append<a id='append'></a>

1. group files according to site name by finding common value from sitelist in `matches` string
2. for each site, select the first shapefile and assign it as base object
3. append all other shapefiles to the base object with `gpd.append()`
4. project crs to wgs84 utm zone 13
4. export the gpdf as a shapefile named: sitelist[i] + '_' + 'TreeStem_pts_WGS84UTM13.shp'

In [16]:
# add full path to all filenames
trees_sf_paths = [renamed_dir + os.sep + i for i in trees_sf]
trees_sf_paths[:5]

['/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM071811A_Snodgrass-2_Student_Project.shp',
 '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/HETTEMAS082510A_PointLookout-1_Student_Project.shp',
 '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM072909A_Snodgrass-1_Student_Project.shp',
 '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM081210A_OhioPass-1_Student_Project.shp',
 '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM081211B_OhioPass-1_Student_Project.shp']

In [17]:
# group files by plot using list comprehension 
trees_sf_grouped = [[s for s in trees_sf_paths if key in s] for key in set(sitelist)]
trees_sf_grouped = [i for i in trees_sf_grouped if len(i) != 0] # filter out a few artifact empty lists

In [18]:
# output list of grouped tree shapefiles in directory
trees_sf_grouped

[['/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM072211A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM081712A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM072407A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM081714A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM072708A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENAMED/WORSHAMM072710A_SplainsGulch-1_Student_Project.shp',
  '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_RENA

In [ ]:
# aggregate, import, and append
alltrees_gpdf = []
for thing in trees_sf_grouped:
    site_gpdf = []
    for i in thing:
        gpdf = gpd.read_file(i)
        site_gpdf.append(gpdf)
    alltrees = site_gpdf[0].append(site_gpdf[1:])
    alltrees.to_crs(epsg = 32613, inplace = True)
    alltrees = alltrees.loc[alltrees.geom_type == 'Point']
    site = [s for s in sitelist if s in thing[0]][0]
    alltrees['Site'] = site
    alltrees.to_file('/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset/Scratch/RMBL_GPS_Data_MERGEDBYPLOT/' + site + '.shp')
    alltrees_gpdf.append(alltrees)